In [5]:
from sklearn.cross_validation import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
import pandas as pd
import numpy as np
import time
import datetime

# Загрузим обе выборки: обучающую и тестовую.
data_train = pd.read_csv('features.csv', index_col='match_id')
data_test = pd.read_csv('features_test.csv', index_col='match_id')

# target_col будет хранить имя признака, который является целевым.
# all_feature_cols будет хранить список всех признаков, которые встречаются в тестовой выборке.
# categorial_cols будет хранить список категориальных признаков
target_col = 'radiant_win'
all_feature_cols = set([col for col in data_test if col != target_col])
categorial_cols = set(['{}{}_hero'.format(team, number) for team in 'rd' for number in range(1, 6)])
categorial_cols.add('lobby_type')

# Заполним пропуски в данных средними значениями колонки.
for dataset in [data_train, data_test]:
    for feature_name in dataset:
        if dataset[feature_name].isnull().sum() > 0:
            dataset[feature_name] = dataset[feature_name].fillna(dataset[feature_name].mean())

# features = data_train[[col for col in data_train if col is not 'radiant_win']]
y = data_train[target_col]

# X - это данные, на которых мы будем обучаться. Заметьте: в них нет категориальных признаков.
X = data_train[list(all_feature_cols - categorial_cols)]

# Сюда мы включим категориальные признаки.
X_categorial = data_train[list(all_feature_cols)]

def run_gradient_boosting(X_=X, n_estimators=100):
    kf = KFold(n=X_.shape[0], n_folds=5, shuffle=True)
    print('n_estimators: {}'.format(n_estimators))

    clf = GradientBoostingClassifier(n_estimators=n_estimators)

    start_time = datetime.datetime.now()
    scores = cross_val_score(clf, X_, y, cv=kf, scoring='roc_auc')
    elapsed_time = datetime.datetime.now() - start_time

    print('cross_val_score: {}'.format(sum(scores) / len(scores)))
    print('corss validation lasted {}'.format(elapsed_time))
    print()


hero_to_number = dict()
s = set()

cnt = 0
for col in categorial_cols:
    if col != 'lobby_type':
        for hero in data_train[col]:
            if hero not in s:
                s.add(hero)
                hero_to_number[hero] = cnt
                cnt += 1

X_pick = np.zeros((X_categorial.shape[0], max(s)))

for i, match_id in enumerate(X_categorial.index):
    for p in range(5):
        X_pick[i, X_categorial.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_categorial.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1


X_bag = np.hstack((X, X_pick))

## Градиентный бустинг по данным с категориальными признаками

In [6]:
run_gradient_boosting(X_=X_categorial)

n_estimators: 100
cross_val_score: 0.7066612287490657
corss validation lasted 0:04:07.896795



## Градиентный бустинг по данным без категориальных признаков

In [7]:
run_gradient_boosting()

n_estimators: 100
cross_val_score: 0.706675084368468
corss validation lasted 0:03:30.107616



## Градиентный бустинг по данным с мешком слов по героям

In [8]:
clf = run_gradient_boosting(X_=X_bag)

n_estimators: 100
cross_val_score: 0.7139306517324352
corss validation lasted 0:12:40.229536

